In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torchsummary import summary

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable


manualSeed = 999
manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
torch.__version__

In [ ]:
PATH = '/content/dataset_org/'
org = '/content/PlantVillage-Dataset/raw/color/'

os.mkdir(PATH)

In [ ]:
from glob import glob
names = [os.path.basename(x) for x in glob(f'{org}Corn*')]
names

In [ ]:
import shutil
import regex as reg
import glob

for k, i in enumerate(glob.iglob(f'{org}Corn*')):
  destination = shutil.copytree(i, PATH+names[k])

In [ ]:
import pathlib

data_dir = pathlib.Path('/content/dataset_org')
print(type(data_dir))
print(data_dir)
image_count = len(list(data_dir.glob('*/*.JPG')))
l = list(data_dir.glob('*/*.JPG'))
print(image_count)

In [ ]:
dataroot = PATH

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 32

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64
image_size256 = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 30

# Learning rate for optimizers
lr = 0.0001

# Beta1 and Beta2 hyperparam for Adam optimizers
beta1 = 0.9
beta2 = 0.99

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 2

In [ ]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
dataset256 = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size256),
                               transforms.CenterCrop(image_size256),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader256 = torch.utils.data.DataLoader(dataset256, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

real_batch = next(iter(dataloader256))
plt.figure(figsize=(16,16))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:

class Generator64(nn.Module):
    def __init__(self, ngpu):
        super(Generator64, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d( nc, ngf , 9, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf , 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d(ngf , ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Conv2d( ngf, ngf * 2, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.Conv2d( ngf * 2, ngf, 3, 1,padding = "same", bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.Conv2d( ngf, nc, 3, 1,padding = "same", bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netG64 = Generator64(ngpu).to(device)
summary(netG64, (3,64,64))

if (device.type == 'cuda') and (ngpu > 1):
    netG64 = nn.DataParallel(netG64, list(range(ngpu)))

netG64.apply(weights_init)

print(netG64)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf*4, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Flatten(1),
            nn.Linear(1,1),
            nn.Sigmoid()

        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netD = Discriminator(ngpu).to(device)
summary(netD, (3,64,64))
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
netD.apply(weights_init)

print(netD)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, 3, 4, 2, 1, bias=False),
            nn.BatchNorm2d(3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netEn = Encoder(ngpu).to(device)
summary(netEn, (3,256,256))
if (device.type == 'cuda') and (ngpu > 1):
    netEn = nn.DataParallel(netEn, list(range(ngpu)))
    
netEn.apply(weights_init)

print(netEn)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, ngpu):
        super(Decoder, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nc, ngf, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.Upsample(scale_factor = 2),
            nn.ConvTranspose2d(ngf, ngf * 2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.Upsample(scale_factor = 2),

            nn.ConvTranspose2d( ngf * 2, ngf * 2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d( ngf * 2, 3, 3, 1, 1, bias=False),
            nn.BatchNorm2d(3),
            nn.ReLU(True),

            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netDec = Decoder(ngpu).to(device)
summary(netDec, (3,64,64))

if (device.type == 'cuda') and (ngpu > 1):
    netDec = nn.DataParallel(netDec, list(range(ngpu)))

netDec.apply(weights_init)

print(netDec)

In [ ]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, nz, 1, 1, device=device)

real_label = 1.
fake_label = 0.

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
from torchvision.models import vgg19

class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        vgg19_model = vgg19(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(vgg19_model.features.children())[:18])

    def forward(self, img):
        return self.feature_extractor(img)

In [ ]:
feature_extractor = FeatureExtractor()
criterion_content = torch.nn.L1Loss()
criterion_content = criterion_content.cuda()


In [ ]:
feature_extractor.eval()

In [ ]:
feature_extractor = feature_extractor.cuda()

In [ ]:
for i, data in enumerate(dataloader256, 0):
  netEn.zero_grad()
  real_cpu = data[0].to(device)
  b_size = real_cpu.size(0)
  label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
  fixed_noise_new = netEn(real_cpu)
  print(fixed_noise_new.shape)

In [ ]:
from torch.autograd import Variable
import torch.autograd as autograd
import math
import sys
import torch.nn.functional as F

cuda = bool(torch.cuda.is_available())
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

In [ ]:

img_list = []
G_losses = []
D_losses = []
iters = 0
lambda_gp = 10

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):

      for j, data256 in enumerate(dataloader256, 0):

        if i==j:
          netD.zero_grad()
          real_cpu = data[0].to(device)
          b_size = real_cpu.size(0)
          label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

          real_cpu256 = data256[0].to(device)
          b_size256 = real_cpu256.size(0)

          encoded = netEn(real_cpu256)


          output = netD(real_cpu).view(-1)
          errD_real = criterion(output, label)
          errD_real.backward()
          D_x = output.mean().item()

          noise = torch.randn(b_size, nz, 1, 1, device=device)
          fake_new = netG64(encoded)

          output_inp = real_cpu256.view(-1)

          gradient_penalty = compute_gradient_penalty(netD, real_cpu.data, fake_new.data)
          label.fill_(fake_label)
          output = netD(fake_new.detach()).view(-1)
          errD_fake = criterion(output, label) + lambda_gp * gradient_penalty
          errD_fake.backward()
          D_G_z1 = output.mean().item()
          errD = errD_real + errD_fake + lambda_gp * gradient_penalty
          optimizerD.step()

          netG64.zero_grad()

          loss_real = criterion(netD(real_cpu), label)
          loss_fake = criterion(netD(fake.detach()), fake_label)


          gen_features = feature_extractor(fake_new)
          real_features = feature_extractor(real_cpu)
          loss_content = criterion_content(gen_features, real_features.detach())


          output = netD(fake_new).view(-1)
          errG = ((1e-1 - 1e-5)/(epoch+1))*criterion(output, label) + loss_content
          errG.backward()

          output_dec = netDec(fake_new).view(-1)
          total_err = criterion(output_dec,output_inp)

          D_G_z2 = output.mean().item()
          optimizerG.step()
          if i % 50 == 0:
              print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f \tTotal_Loss: %.4f'
                    % (epoch, num_epochs, i, len(dataloader),
                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2, total_err))
          
          G_losses.append(errG.item())
          D_losses.append(errD.item())
          
          if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
              with torch.no_grad():
                  fake = netG64(fixed_noise_new).detach().cpu()
              img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
              
          iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
real_batch = next(iter(dataloader))

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
GEN_PATH = '/content/Gen_images/'

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

gen_images = []
for img_path in glob.glob(GEN_PATH+'*.png'):
    print(img_path)
    gen_images.append(mpimg.imread(img_path))

columns = 20
for i, image in enumerate(gen_images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.axis('off')
    plt.imshow(image)

In [ ]:
dataset256 = dset.ImageFolder(root=GEN_PATH,
                           transform=transforms.Compose([
                               transforms.Resize(image_size256),
                               transforms.CenterCrop(image_size256),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
dataloader256 = torch.utils.data.DataLoader(dataset256, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

real_batch = next(iter(dataloader256))

In [ ]:
num_classes = 4
total_size = 14864
train_size = math.ceil(0.7 * total_size)
test_size = total_size - train_size

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataloader256, [train_size, test_size])

In [ ]:
class Classifier(nn.Module):
    def __init__(self, ngpu):
        super(Classifier, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.Conv2d(nc, 16, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(16, 8, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(8, 8, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(8, 8, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(8, 6, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(6, 16, 3, 1, 1, bias=False),
            nn.ReLU(0.1),
            nn.MaxPool2d(3),
            nn.Conv2d(16, 16, 4, 1, 0, bias=False),
            nn.Flatten(1),
            nn.Linear(64,32),
            nn.Linear(32,num_classes),
            nn.Softmax()

        )

    def forward(self, input):
        return self.main(input)


In [ ]:
clgannet = Classifier(ngpu).to(device)
summary(clgannet, (3,256,256))
if (device.type == 'cuda') and (ngpu > 1):
    clgannet = nn.DataParallel(clgannet, list(range(ngpu)))
    
print(clgannet)

In [ ]:

optimizer = torch.optim.Adam(clgannet.parameters(), lr = lr,betas=(beta1,beta2), momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(num_epochs):  

    running_loss = 0.0
    for i, data in enumerate(train_dataset, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = clgannet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
MODEL_PATH = './clgannet.pth'
torch.save(clgannet.state_dict(), MODEL_PATH)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataset:
        images, labels = data
        outputs = clgannet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
